In [3]:
import numpy as np
import pandas as pd
import ast

In [4]:
dress_df = pd.read_csv('/content/drive/MyDrive/Stylumia/Dataset/dresses_bd_processed_data.csv')
jeans_df = pd.read_csv('/content/drive/MyDrive/Stylumia/Dataset/jeans_bd_processed_data.csv')

In [8]:
dress_df.head()

,selling_price,discount,category_id,meta_info,product_id,pdp_url,sku,brand,department_id,last_seen_date,launch_on,mrp,product_name,feature_image_s3,channel_id,feature_list,description,style_attributes,pdp_images_s3
0,{'INR': 474848.9539},0.0,30,Slim fit. Designed to hit at the ankle. UK siz...,b613d7b5dfe86f3e695d931d31fd729fdf44e181f14079...,https://www.ralphlauren.co.uk/en/kristian-silk...,479495,RALPH LAUREN,2,2025-05-01,2020-02-29,{'INR': 474848.9539},Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{},['https://gallery.stylumia.com/originals/2020/...
1,{'INR': 464648.6919},0.0,30,Slim fit. Designed to hit at the ankle. UK siz...,482b10a23f8d00cfc7c9bbeeac4e26d25dd303d8e62e97...,https://www.ralphlauren.co.uk/en/kristian-silk...,502670,RALPH LAUREN,2,2025-05-08,2020-02-29,{'INR': 464648.6919},Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{},['https://gallery.stylumia.com/originals/2020/...
2,{'INR': 29496.0812},0.0,30,Fit-and-flare silhouette. Intended to hit at t...,3508b052ef7a5eea820423b97713612bc92a3f2301a3d3...,https://www.ralphlauren.co.uk/en/fit-and-flare...,478766,RALPH LAUREN,2,2025-05-08,2020-02-29,{'INR': 29496.0812},Fit-and-Flare Shirtdress,https://gallery.stylumia.com/originals/2020/02...,14,['Fit-and-flare silhouette. Intended to hit at...,Airy georgette and a flattering fit-and-flare ...,{},['https://gallery.stylumia.com/originals/2020/...
3,{'INR': 17156.9392},0.0,30,"Fits true to size; take your normal size, Stra...",6360245240b68885bd4dbcef8d8856c0fb13f1314769f5...,https://www.anthropologie.com/shop/adena-crepe...,50297209_041,BHLDN,2,2025-01-31,2020-05-27,{'INR': 17156.9392},Adena Crepe Dress,https://gallery.stylumia.com/originals/2020/05...,48,"['Back zip', 'Polyester; polyester lining', 'P...","A sleek square neckline tops this stretchy, bo...","{'modelNotes': '', 'dimensions': 'Fits true to...",['https://gallery.stylumia.com/originals/2020/...
4,{'INR': 26079.5467},0.0,30,Fit-and-flare silhouette Designed to hit at th...,5d07037957e64d1e218499cb7d7a8e5e57aa59249bb806...,https://www.ralphlauren.co.uk/en/belted-cotton...,478750,RALPH LAUREN,2,2025-05-08,2021-02-12,{'INR': 26079.5467},Belted Cotton-Blend Shirtdress,https://gallery.stylumia.com/originals/2021/02...,14,['Fit-and-flare silhouette Designed to hit at ...,This iteration of Lauren's iconic shirtdress i...,{},['https://gallery.stylumia.com/originals/2021/...


In [35]:
# Robust function to extract price for dresses
def extract_price_dress(value):
    try:
        # If it's a string, e.g., "{'INR': 1234.56}"
        if isinstance(value, str) and "'INR':" in value:
            # Remove the prefix and suffix to isolate the number
            price_str = value.split(':')[1].replace('}', '').strip()
            return float(price_str)

        # If it's already a dict (rare, but just in case)
        if isinstance(value, dict):
            return value.get('INR', None)

    except Exception as e:
        return None

    return None

In [36]:
# Robust function to extract price for jeans
def extract_price_jeans(value):
    try:
        # If it's a string, e.g., "{'INR': 1234.56}"
        if isinstance(value, str) and "'USD':" in value:
            # Remove the prefix and suffix to isolate the number
            price_str = value.split(':')[1].replace('}', '').strip()
            return float(price_str)

        # If it's already a dict (rare, but just in case)
        if isinstance(value, dict):
            return value.get('USD', None)

    except Exception as e:
        return None

    return None

In [37]:
# Define cleaning function for the dress DataFrame
def clean_df_dress(df):
    # Clean price columns
    df['selling_price'] = df['selling_price'].apply(extract_price_dress)
    df['mrp'] = df['mrp'].apply(extract_price_dress)

    # Drop unwanted columns
    drop_columns = [
        'category_id', 'pdp_url', 'pdp_images_s3',
        'meta_info', 'department_id', 'last_seen_date', 'launch_on'
    ]
    df.drop(columns=drop_columns, inplace=True, errors='ignore')

    return df

In [38]:
# Define cleaning function for the jeans DataFrame
def clean_df_jeans(df):
    # Clean price columns
    df['selling_price'] = df['selling_price'].apply(extract_price_jeans)
    df['mrp'] = df['mrp'].apply(extract_price_jeans)

    # Drop unwanted columns
    drop_columns = [
        'category_id', 'pdp_url', 'pdp_images_s3',
        'meta_info', 'department_id', 'last_seen_date', 'launch_on'
    ]
    df.drop(columns=drop_columns, inplace=True, errors='ignore')

    return df

In [39]:
# Clean both dataframes
clean_dress = clean_df_dress(dress_df.copy())
clean_jeans = clean_df_jeans(jeans_df.copy())

In [43]:
# Save cleaned DataFrames
clean_dress.to_csv('/content/drive/MyDrive/Stylumia/Dataset/dress_clean.csv', index=False)
clean_jeans.to_csv('/content/drive/MyDrive/Stylumia/Dataset/jeans_clean.csv', index=False)

In [44]:
clean_dress.head()

,selling_price,discount,product_id,sku,brand,mrp,product_name,feature_image_s3,channel_id,feature_list,description,style_attributes
0,474848.9539,0.0,b613d7b5dfe86f3e695d931d31fd729fdf44e181f14079...,479495,RALPH LAUREN,474848.9539,Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{}
1,464648.6919,0.0,482b10a23f8d00cfc7c9bbeeac4e26d25dd303d8e62e97...,502670,RALPH LAUREN,464648.6919,Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{}
2,29496.0812,0.0,3508b052ef7a5eea820423b97713612bc92a3f2301a3d3...,478766,RALPH LAUREN,29496.0812,Fit-and-Flare Shirtdress,https://gallery.stylumia.com/originals/2020/02...,14,['Fit-and-flare silhouette. Intended to hit at...,Airy georgette and a flattering fit-and-flare ...,{}
3,17156.9392,0.0,6360245240b68885bd4dbcef8d8856c0fb13f1314769f5...,50297209_041,BHLDN,17156.9392,Adena Crepe Dress,https://gallery.stylumia.com/originals/2020/05...,48,"['Back zip', 'Polyester; polyester lining', 'P...","A sleek square neckline tops this stretchy, bo...","{'modelNotes': '', 'dimensions': 'Fits true to..."
4,26079.5467,0.0,5d07037957e64d1e218499cb7d7a8e5e57aa59249bb806...,478750,RALPH LAUREN,26079.5467,Belted Cotton-Blend Shirtdress,https://gallery.stylumia.com/originals/2021/02...,14,['Fit-and-flare silhouette Designed to hit at ...,This iteration of Lauren's iconic shirtdress i...,{}
